# Working with Market Depth and Trades

## Display 3-depth

In [1]:
from numba import njit

@njit
def print_3depth(hbt):
    while hbt.elapse(60 * 1e9) == 0:
        print('current_timestamp:', hbt.current_timestamp)

        # Gets the market depth for the first asset, in the same order as when you created the backtest.
        depth = hbt.depth(0)

        # a key of bid_depth or ask_depth is price in ticks.
        # (integer) price_tick = price / tick_size
        i = 0
        for price_tick in range(depth.best_ask_tick, depth.best_ask_tick + 100):
            qty = depth.ask_qty_at_tick(price_tick)
            if qty > 0:
                print(
                    'ask: ',
                    qty,
                    '@',
                    np.round(price_tick * depth.tick_size, 1)
                )
                
                i += 1
                if i == 3:
                    break
        i = 0
        for price_tick in range(depth.best_bid_tick, max(depth.best_bid_tick - 100, 0), -1):
            qty = depth.bid_qty_at_tick(price_tick)
            if qty > 0:
                print(
                    'bid: ',
                    qty,
                    '@',
                    np.round(price_tick * depth.tick_size, 1)
                )
            
                i += 1
                if i == 3:
                    break
    return True

In [2]:
import numpy as np

btcusdt_20240719 = np.load('usdm/btcusdt_20240719.npz')['data']
btcusdt_20240719_eod = np.load('usdm/btcusdt_20240718_eod.npz')['data']

In [3]:
from hftbacktest import BacktestAsset, HashMapMarketDepthBacktest

asset = (
    BacktestAsset()
        .data(btcusdt_20240719)
        .initial_snapshot(btcusdt_20240719_eod)
        .linear_asset(1.0) 
        .constant_latency(10_000_000, 10_000_000)
        .risk_adverse_queue_model() 
        .no_partial_fill_exchange()
        .trading_value_fee_model(0.0002, 0.0007)
        .tick_size(0.1)
        .lot_size(0.001)
)

hbt = HashMapMarketDepthBacktest([asset])

print_3depth(hbt)

_ = hbt.close()

current_timestamp: 1721347261533000000
ask:  1.661 @ 63980.0
ask:  0.166 @ 63980.1
ask:  0.002 @ 63981.4
bid:  17.429 @ 63979.9
bid:  0.011 @ 63979.8
bid:  0.347 @ 63979.7
current_timestamp: 1721347321533000000
ask:  40.245 @ 63931.4
ask:  1.133 @ 63931.5
ask:  0.1 @ 63931.6
bid:  2.656 @ 63931.3
bid:  0.166 @ 63930.7
bid:  0.003 @ 63930.6
current_timestamp: 1721347381533000000
ask:  1.147 @ 63870.9
ask:  0.002 @ 63871.2
ask:  0.166 @ 63871.6
bid:  10.989 @ 63870.8
bid:  0.278 @ 63870.7
bid:  0.007 @ 63870.6
current_timestamp: 1721347441533000000
ask:  15.426 @ 63848.5
ask:  0.009 @ 63848.6
ask:  0.002 @ 63849.0
bid:  11.01 @ 63848.4
bid:  0.004 @ 63848.3
bid:  1.083 @ 63848.0
current_timestamp: 1721347501533000000
ask:  9.029 @ 63889.1
ask:  0.013 @ 63889.2
ask:  0.005 @ 63889.3
bid:  5.758 @ 63889.0
bid:  0.097 @ 63888.9
bid:  0.114 @ 63888.0


## Efficient Market Depth Access

`ROIVectorMarketDepth` provides more efficient market depth access through a vector that holds a limited price range of interest. The backtester using this feature can be created by `ROIVectorMarketDepthBacktest`.

In [4]:
from numba import njit

@njit
def print_3depth_fast(hbt):
    roi_lb_tick = int(round(30000 / 0.1))
    roi_ub_tick = int(round(90000 / 0.1))
    
    while hbt.elapse(60 * 1e9) == 0:
        print('current_timestamp:', hbt.current_timestamp)

        # Gets the market depth for the first asset, in the same order as when you created the backtest.
        depth = hbt.depth(0)

        # a key of bid_depth or ask_depth is price in ticks.
        # (integer) price_tick = price / tick_size
        i = 0
        for price_tick in range(depth.best_ask_tick, depth.best_ask_tick + 100):
            # depth.ask_depth returns the ask depth array, whose length is (roi_ub_tick + 1 - roi_lb_tick),
            # containing the quantities ranging from roi_lb_tick to roi_ub_tick.
            # Checks that the price_tick is in that range and adjust the index by subtracting roi_lb_tick.
            if price_tick < roi_lb_tick or price_tick > roi_ub_tick:
                continue
            t = price_tick - roi_lb_tick
            qty = depth.ask_depth[t]
            if qty > 0:
                print(
                    'ask: ',
                    qty,
                    '@',
                    np.round(price_tick * depth.tick_size, 1)
                )
                
                i += 1
                if i == 3:
                    break
        i = 0
        for price_tick in range(depth.best_bid_tick, max(depth.best_bid_tick - 100, 0), -1):
            # depth.bid_depth returns the bid depth array, whose length is (roi_ub_tick + 1 - roi_lb_tick),
            # containing the quantities ranging from roi_lb_tick to roi_ub_tick.
            # Checks that the price_tick is in that range and adjust the index by subtracting roi_lb_tick.
            if price_tick < roi_lb_tick or price_tick > roi_ub_tick:
                continue
            t = price_tick - roi_lb_tick
            qty = depth.bid_depth[t]
            if qty > 0:
                print(
                    'bid: ',
                    qty,
                    '@',
                    np.round(price_tick * depth.tick_size, 1)
                )
            
                i += 1
                if i == 3:
                    break
    return True

In [5]:
from hftbacktest import ROIVectorMarketDepthBacktest

asset = (
    BacktestAsset()
        .data(btcusdt_20240719)
        .initial_snapshot(btcusdt_20240719_eod)
        .linear_asset(1.0) 
        .constant_latency(10_000_000, 10_000_000)
        .risk_adverse_queue_model() 
        .no_partial_fill_exchange()
        .trading_value_fee_model(0.0002, 0.0007)
        .tick_size(0.1)
        .lot_size(0.001)
        # Sets the lower bound price for the range of interest in the market depth.
        .roi_lb(30000)
        # Sets the upper bound price for the range of interest in the market depth.
        .roi_ub(90000)
)

hbt = ROIVectorMarketDepthBacktest([asset])

print_3depth_fast(hbt)

_ = hbt.close()

current_timestamp: 1721347261533000000
ask:  1.661 @ 63980.0
ask:  0.166 @ 63980.1
ask:  0.002 @ 63981.4
bid:  17.429 @ 63979.9
bid:  0.011 @ 63979.8
bid:  0.347 @ 63979.7
current_timestamp: 1721347321533000000
ask:  40.245 @ 63931.4
ask:  1.133 @ 63931.5
ask:  0.1 @ 63931.6
bid:  2.656 @ 63931.3
bid:  0.166 @ 63930.7
bid:  0.003 @ 63930.6
current_timestamp: 1721347381533000000
ask:  1.147 @ 63870.9
ask:  0.002 @ 63871.2
ask:  0.166 @ 63871.6
bid:  10.989 @ 63870.8
bid:  0.278 @ 63870.7
bid:  0.007 @ 63870.6
current_timestamp: 1721347441533000000
ask:  15.426 @ 63848.5
ask:  0.009 @ 63848.6
ask:  0.002 @ 63849.0
bid:  11.01 @ 63848.4
bid:  0.004 @ 63848.3
bid:  1.083 @ 63848.0
current_timestamp: 1721347501533000000
ask:  9.029 @ 63889.1
ask:  0.013 @ 63889.2
ask:  0.005 @ 63889.3
bid:  5.758 @ 63889.0
bid:  0.097 @ 63888.9
bid:  0.114 @ 63888.0


## Order Book Imbalance

In [6]:
@njit
def orderbookimbalance(hbt, out):
    roi_lb_tick = int(round(30000 / 0.1))
    roi_ub_tick = int(round(90000 / 0.1))

    while hbt.elapse(10 * 1e9) == 0:
        depth = hbt.depth(0)
        
        mid_price = (depth.best_bid + depth.best_ask) / 2.0
        
        sum_ask_qty_50bp = 0.0
        sum_ask_qty = 0.0
        for price_tick in range(depth.best_ask_tick, roi_ub_tick + 1):
            if price_tick < roi_lb_tick or price_tick > roi_ub_tick:
                continue
            t = price_tick - roi_lb_tick
            
            ask_price = price_tick * depth.tick_size
            depth_from_mid = (ask_price - mid_price) / mid_price
            if depth_from_mid > 0.01:
                break
            sum_ask_qty += depth.ask_depth[t]
            
            if depth_from_mid <= 0.005:
                sum_ask_qty_50bp = sum_ask_qty
                
                
        sum_bid_qty_50bp = 0.0
        sum_bid_qty = 0.0
        for price_tick in range(depth.best_bid_tick, roi_lb_tick - 1, -1):
            if price_tick < roi_lb_tick or price_tick > roi_ub_tick:
                continue
            t = price_tick - roi_lb_tick
            
            bid_price = price_tick * depth.tick_size
            depth_from_mid = (mid_price - bid_price) / mid_price
            if depth_from_mid > 0.01:
                break
            sum_bid_qty += depth.bid_depth[t]
            
            if depth_from_mid <= 0.005:
                sum_bid_qty_50bp = sum_bid_qty
                
        imbalance_50bp = sum_bid_qty_50bp - sum_ask_qty_50bp
        imbalance_1pct = sum_bid_qty - sum_ask_qty
        imbalance_tob = depth.bid_depth[depth.best_bid_tick - roi_lb_tick] - depth.ask_depth[depth.best_ask_tick - roi_lb_tick]
        
        out.append((hbt.current_timestamp, imbalance_tob, imbalance_50bp, imbalance_1pct))        
    return True

In [7]:
from numba.typed import List
from numba.types import Tuple, float64

hbt = ROIVectorMarketDepthBacktest([asset])

tup_ty = Tuple((float64, float64, float64, float64))
out = List.empty_list(tup_ty, allocated=100_000)

orderbookimbalance(hbt, out)

_ = hbt.close()

In [8]:
import polars as pl

df = pl.DataFrame(out).transpose()
df.columns = ['Local Timestamp', 'TOB Imbalance', '0.5% Imbalance', '1% Imbalance']
df = df.with_columns(
    pl.from_epoch('Local Timestamp', time_unit='ns')
)

df

Local Timestamp,TOB Imbalance,0.5% Imbalance,1% Imbalance
datetime[ns],f64,f64,f64
2024-07-19 00:00:11.532999936,-43.71,-137.642,-155.253
2024-07-19 00:00:21.532999936,-4.954,-77.894,-112.888
2024-07-19 00:00:31.532999936,-19.715,-85.918,-117.391
2024-07-19 00:00:41.532999936,-16.019,-86.89,-139.724
2024-07-19 00:00:51.532999936,-11.019,-89.906,-216.074
…,…,…,…
2024-07-19 00:04:21.532999936,0.598,55.877,-112.652
2024-07-19 00:04:31.532999936,23.956,64.729,-175.977
2024-07-19 00:04:41.532999936,11.589,-58.843,-241.947


In [9]:
df.plot(x='Local Timestamp')

:NdOverlay   [Variable]
   :Curve   [Local Timestamp]   (value)

## Display last trades between the step

In [10]:
from hftbacktest import BUY_EVENT

@njit
def print_trades(hbt):
    while hbt.elapse(60 * 1e9) == 0:
        print('-------------------------------------------------------------------------------')
        print('current_timestamp:', hbt.current_timestamp)

        # Gets the last trades occurring in the market, not the trades of our orders.
        last_trades = hbt.last_trades(0)
        
        num = 0
        for last_trade in last_trades:
            if num > 10:
                print('...')
                break
            print(
                'exch_timestamp:',
                last_trade.exch_ts,
                'buy' if (last_trade.ev & BUY_EVENT) == BUY_EVENT else 'sell',
                last_trade.qty,
                '@',
                last_trade.px
            )
            num += 1

        # To prevent accumulating all last trades, which may cause a slowdown,
        # clear_last_trades needs to be called.
        # After this, accessing `last_trades` will cause a crash.
        hbt.clear_last_trades(0)
    return True

In [11]:
asset = (
    BacktestAsset()
        .data(btcusdt_20240719)
        .initial_snapshot(btcusdt_20240719_eod)
        .linear_asset(1.0) 
        .constant_latency(10_000_000, 10_000_000)
        .risk_adverse_queue_model() 
        .no_partial_fill_exchange()
        .trading_value_fee_model(0.0002, 0.0007)
        .tick_size(0.1)
        .lot_size(0.001)
        # To retrieve the last trades, `last_trades_capacity` should be set.
        .last_trades_capacity(1000)
        .roi_lb(30000)
        .roi_ub(90000)
)

hbt = ROIVectorMarketDepthBacktest([asset])

print_trades(hbt)

_ = hbt.close()

-------------------------------------------------------------------------------
current_timestamp: 1721347261533000000
exch_timestamp: 1721347202812000000 buy 0.31 @ 63960.0
exch_timestamp: 1721347202824000000 sell 0.003 @ 63959.9
exch_timestamp: 1721347202834000000 sell 0.015 @ 63959.9
exch_timestamp: 1721347202847000000 sell 0.01 @ 63959.9
exch_timestamp: 1721347202849000000 buy 0.026 @ 63960.0
exch_timestamp: 1721347202849000000 buy 0.054 @ 63960.0
exch_timestamp: 1721347202849000000 buy 0.064 @ 63960.0
exch_timestamp: 1721347202849000000 buy 0.024 @ 63960.0
exch_timestamp: 1721347202849000000 buy 0.028 @ 63960.0
exch_timestamp: 1721347202849000000 buy 0.036 @ 63960.0
exch_timestamp: 1721347202849000000 buy 0.062 @ 63960.0
...
-------------------------------------------------------------------------------
current_timestamp: 1721347321533000000
exch_timestamp: 1721347261567000000 buy 0.155 @ 63980.0
exch_timestamp: 1721347261567000000 buy 0.002 @ 63980.0
exch_timestamp: 1721347261567

## Rolling Volume-Weighted Average Price

In [12]:
@njit
def rolling_vwap(hbt, out):
    buy_amount_bin = np.zeros(100_000, np.float64)
    buy_qty_bin = np.zeros(100_000, np.float64)
    sell_amount_bin = np.zeros(100_000, np.float64)
    sell_qty_bin = np.zeros(100_000, np.float64)
    
    idx = 0
    last_trade_price = np.nan
    
    while hbt.elapse(10 * 1e9) == 0:
        last_trades = hbt.last_trades(0)
        
        for last_trade in last_trades:
            if (last_trade.ev & BUY_EVENT) == BUY_EVENT:
                buy_amount_bin[idx] += last_trade.px * last_trade.qty
                buy_qty_bin[idx] += last_trade.qty
            else:
                sell_amount_bin[idx] += last_trade.px * last_trade.qty
                sell_qty_bin[idx] += last_trade.qty
                
        hbt.clear_last_trades(0)
        idx += 1

        if idx >= 1:
            vwap10sec = np.divide(
                buy_amount_bin[idx - 1] + sell_amount_bin[idx - 1], 
                buy_qty_bin[idx - 1] + sell_qty_bin[idx - 1]
            )
        else:
            vwap10sec = np.nan
        
        if idx >= 6:
            vwap1m = np.divide(
                np.sum(buy_amount_bin[idx - 6:idx]) + np.sum(sell_amount_bin[idx - 6:idx]), 
                np.sum(buy_qty_bin[idx - 6:idx]) + np.sum(sell_qty_bin[idx - 6:idx])
            )
            buy_vwap1m = np.divide(np.sum(buy_amount_bin[idx - 6:idx]), np.sum(buy_qty_bin[idx - 6:idx]))
            sell_vwap1m = np.divide(np.sum(sell_amount_bin[idx - 6:idx]), np.sum(sell_qty_bin[idx - 6:idx]))
        else:
            vwap1m = np.nan
            buy_vwap1m = np.nan
            sell_vwap1m = np.nan
        
        out.append((hbt.current_timestamp, vwap10sec, vwap1m, buy_vwap1m, sell_vwap1m))
    return True

In [13]:
hbt = ROIVectorMarketDepthBacktest([asset])

tup_ty = Tuple((float64, float64, float64, float64, float64))
out = List.empty_list(tup_ty, allocated=100_000)

rolling_vwap(hbt, out)

_ = hbt.close()

In [14]:
df = pl.DataFrame(out).transpose()
df.columns = ['Local Timestamp', '10-sec VWAP', '1-min VWAP', '1-min Buy VWAP', '1-min Sell VWAP']
df = df.with_columns(
    pl.from_epoch('Local Timestamp', time_unit='ns')
)

df

Local Timestamp,10-sec VWAP,1-min VWAP,1-min Buy VWAP,1-min Sell VWAP
datetime[ns],f64,f64,f64,f64
2024-07-19 00:00:11.532999936,63945.118216,NaN,NaN,NaN
2024-07-19 00:00:21.532999936,63922.02134,NaN,NaN,NaN
2024-07-19 00:00:31.532999936,63916.801405,NaN,NaN,NaN
2024-07-19 00:00:41.532999936,63918.503714,NaN,NaN,NaN
2024-07-19 00:00:51.532999936,63963.603699,NaN,NaN,NaN
…,…,…,…,…
2024-07-19 00:04:21.532999936,63854.433462,63858.545404,63862.645421,63856.941675
2024-07-19 00:04:31.532999936,63860.331773,63857.90228,63862.226116,63855.770468
2024-07-19 00:04:41.532999936,63879.192019,63858.85963,63865.370927,63854.657886


In [15]:
df.plot(x='Local Timestamp')

:NdOverlay   [Variable]
   :Curve   [Local Timestamp]   (value)